## Import packages

In [ ]:
import json
import numpy as np
import tensorflow as tf

from tensorflow.keras import Sequential, layers
from tensorflow.keras.losses import CategoricalCrossentropy

from os import listdir
from os.path import join

# Import variables and functions from my own scripts
from functions import f1
from load_features import (
    test_features_AW2,
    features_reshaper,
)

# Limit GPU memory usage
for device in tf.config.experimental.list_physical_devices("GPU"):
    tf.config.experimental.set_memory_growth(device, True)

## Create functions

In [ ]:
# Function which writes the predictions to text files for each video
def write_predictions(test_filenames, test_pred, video_shapes, model):
    # Create path for predictions depending on model
    if model == 'LSTM':
        path = "data/AW2_test_preds/LSTM/"
    elif model == 'FW-RNN':
        path = "data/AW2_test_preds/FW-RNN/"
    
    # Puts all test_video_names from test set in list
    test_videos = []
    for fn in test_filenames:
        video, frame_n = fn.split("\\")
        test_videos.append(video)
    test_videos = list(set(test_videos))

    # Create text file for each video with first line set to all classes in text
    for video in test_videos:
        with open(join(path, video + ".txt"), "w") as fp:
            fp.write("Neutral,Anger,Disgust,Fear,Happiness,Sadness,Surprise")

            
    previous_video = ""
    previous_frame_n = -1
    current_frame_n = 0
    
    def prev_video_append():
        # Function to append missing frames at the end of previous video
        fullvideo = previous_video.replace("_left", "").replace("_right", "")
        video_length = video_shapes.get(fullvideo)
        video_length = video_length[0]

        print(f"Previous video: {previous_video}, diff: {video_length - previous_frame_n}")

        if previous_frame_n != video_length:
            diff = video_length - previous_frame_n
            with open(
                join(path, previous_video + ".txt"), "a"
            ) as fp:
                for i in range(diff):
                    fp.write("\n" + "-1")
    

    for fn, label in zip(test_filenames, test_pred):
        # Convert label to string
        label = str(label)

        # Split filename into videoname and frame_n
        video, frame_n = fn.split("\\")
        frame_n = int(frame_n.strip(".jpg"))

        previous_frame_n = current_frame_n
        current_frame_n = frame_n

        # If we moved to the next video, check if the previous videos frame_n was
        # equal to the total amount of frames of the video. If it wasn't, write to
        # all the missing lines -1
        if previous_video != video:
            if previous_video != "":
                prev_video_append()
            previous_video = video
            previous_frame_n = 0
            
        # Calculate difference between current frame_n and previous one
        diff = current_frame_n - previous_frame_n

        # If frame difference is only 1, then simply write the label on the next line
        if diff == 1:
            with open(join(path, video + ".txt"), "a") as fp:
                fp.write("\n" + label)
        # If the frame difference is larger than 1, simply fill the missing lines with -1, and then write the label
        else:
            with open(join(path, video + ".txt"), "a") as fp:
                for i in range(diff - 1):
                    fp.write("\n" + "-1")
                fp.write("\n" + label)
    
    # Append last video
    previous_frame_n = current_frame_n
    prev_video_append()
    

## Reshape features in sequences and load filenames and videoshapes

In [ ]:
length = 120
seq_test_features = features_reshaper(test_features_AW2, length)

In [ ]:
# Read test set frames filenames
with open("data/filenames/test_filenames_AW2.txt", "r") as fp:
    test_filenames = fp.read().splitlines()

# Read AW2 video shapes to grab the length of each video
with open("data/AW2_video_shapes_woext.json", "r") as fp:
    AW2_video_shapes = json.load(fp)

# LSTM - Predict

In [ ]:
# Load best LSTM model
best_LSTM = tf.keras.models.load_model(
    filepath="data/models/LSTM_model.h5", custom_objects={"F1-metric": f1}, compile=False
)

In [ ]:
# Do predictions on test set
test_pred_LSTM = best_LSTM.predict(seq_test_features, verbose=0)

# Reshape back to (frame, label)
test_pred_LSTM = np.reshape(
    test_pred_LSTM, (test_pred_LSTM.shape[0] * test_pred_LSTM.shape[1], test_pred_LSTM.shape[2])
)

# Convert one hot encoding to integers
test_pred_LSTM = np.argmax(test_pred_LSTM, axis=1)

In [ ]:
# Run write_predictions function for test set predictions with LSTM model
write_predictions(test_filenames, test_pred_LSTM, AW2_video_shapes, 'LSTM')

# FW-RNN - Predict

In [ ]:
# Load best FW-RNN model
best_FWRNN = tf.keras.models.load_model(
    filepath="data/models/FW-RNN_model.h5", custom_objects={"F1-metric": f1}, compile=False
)

In [ ]:
# Do predictions on test set
test_pred_FWRNN = best_FWRNN.predict(seq_test_features, verbose=0)

# Reshape back to (frame, label)
test_pred_FWRNN = np.reshape(
    test_pred_FWRNN, (test_pred_FWRNN.shape[0] * test_pred_FWRNN.shape[1], test_pred_FWRNN.shape[2])
)

# Convert one hot encoding to integers
test_pred_FWRNN = np.argmax(test_pred_FWRNN, axis=1)

In [ ]:
# Run write_predictions function for test set predictions with LSTM model
write_predictions(test_filenames, test_pred_FWRNN, AW2_video_shapes, 'FW-RNN')